# Creación de conversaciones TCP

### Descripción: 
Con este código se va a intentar crear todos los pasos de una conversación TCP, mediante IA generativa. Se pretende pasar una descripción de esta conversación al modelo y que este sea capaz de crear todos los pasos/paquetes necesarios para replicar una conversación con esas mismas especificaciones

In [4]:
import random
import re
import pickle

import openai
openai.api_key = "FILL"

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.l2 import Ether, ARP
from scapy.layers.inet import IP, TCP, ICMP

openai.__version__

C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


'1.10.0'

## Generación de la descipción de una comunicación TCP

<center><img src="https://notes.shichao.io/tcpv1/figure_13-1.png">
    
<center> Esta imagen es genérica, se intentará crear tanto la parte de hanshake como la terminación de la conversación en una primera aproximación.


In [8]:
query1 = "This is a IPv4 TCP conversation summary:\n"
query1 += 'Source: IP="193.24.227.230", port=40301, Window: 4128 // Destination: IP="9.9.9.9", port=53, Window: 256// Others: \n\n'

query1 += "This is the python code to generate the 3 way hanshake packets of the conversation, respectively:\n"
query1 += 'scapy.IP(src="193.24.227.230", dst="9.9.9.9")/scapy.TCP(sport=40301, dport=53, flags="S", seq=x, window=4128)\n'
query1 += 'scapy.IP(src="9.9.9.9", dst="193.24.227.230")/scapy.TCP(sport=53, dport=40301, flags="SA", seq=x+1, ack=y, window=256)\n'
query1 += 'scapy.IP(src="193.24.227.230", dst="9.9.9.9")/scapy.TCP(sport=40301, dport=53, flags="SA", seq=y+1, ack=x+1, window=4128)\n\n'

query1 += "This part between the handshake and the teardown must be filled with bidirectional realistic TCP traffic, here is an example of these packets:\n"
query1 += 'scapy.IP(src="193.24.227.230", dst="9.9.9.9")/scapy.TCP(sport=40301, dport=53, flags="PA", seq=y+1, ack=x+1, window=8192)/payload1\n'
query1 += 'scapy.IP(src="9.9.9.9", dst="193.24.227.230")/scapy.TCP(sport=53, dport=40301, flags="PA", seq=x+1, ack=x+payload1(B)+1, window=16384)/payload2\n'
query1 += 'scapy.IP(src"193.24.227.230", dst="9.9.9.9")/scapy.TCP(sport=40301, dport=53, flags="PA", seq=x+payload1(B)+1, ack=x+payload2(B)+1, window=8192)/payload3\n'
query1 += 'scapy.IP(src="9.9.9.9", dst="193.24.227.230")/scapy.TCP(sport=53, dport=40301, flags="PA", seq=x+payload2(B)+1, ack=x+payload3(B)+1, window=16384)/payload4\n\n'

query1 += "This is the python code to generate the TCP teardown packets of the conversation, respectively:\n"
query1 += 'scapy.IP(src="193.24.227.230", dst="9.9.9.9")/scapy.TCP(sport=40301, dport=53, flags="FA", seq=K, ack=L window=4128)\n'
query1 += 'scapy.IP(src="9.9.9.9", dst="193.24.227.230")/scapy.TCP(sport=53, dport=40301, flags="A", seq=L, ack=K+1, window=256)\n'
query1 += 'scapy.IP(src="9.9.9.9", dst="193.24.227.230")/scapy.TCP(sport=53, dport=40301, flags="FA", seq=L, ack=K+1, window=256)\n'
query1 += 'scapy.IP(src="193.24.227.230", dst="9.9.9.9")/scapy.TCP(sport=40301, dport=53, flags="A", seq=K, ack=L+1 window=4128)\n\n'

query1 += 'Substitute variables x, y with random numbers but they must be the same along the conversation\n'
query1 += 'Variables K and L are critical and must be continuous with seq and ack numbers before them\n'
query1 += 'Create commands for bidirectional realistic TCP packets for a connection between the handshake and the teardown (random number of packets between 5 and 10 for this part), this packets must have payload\n\n'

query1 += 'Pay special attention to the sequence and ack numbers so they make sense for a TCP conversation. The must be continuous throgh the conversation. Here you have a little explanation of how they work:\n'
query1 += 'The Sequence Number (SEQ) marks the starting byte of each segment sent. It increments by the number of bytes in the payload of each outgoing segment, reflecting the continuous flow of data.\n'
query1 += 'The Acknowledgment Number (ACK) acknowledges the receipt of data. It corresponds to the next expected byte the receiver anticipates from the sender. As segments arrive at the receiver, the ACK number adjusts to reflect the highest contiguous sequence number received.\n'
query1 += 'So, SEQ numbers increase with outgoing data, while ACK numbers advance with incoming acknowledgments, ensuring synchronized data exchange in a TCP connection.\n\n'

query1 += "Take your time (few seconds) to generate the commands and compare your proposed solution with real TCP functioning, if anything is different, change it in order to do as real TCP\n\n"

## Carga de posibles conversaciones

In [60]:
with open("../../data/Conversations/TCP/Conv_summaries.txt","r",encoding="utf8") as f:
    Conv_summary = f.read().splitlines()

### Carga de posibles conversaciones anteriores para concatenarlas

In [63]:
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

In [65]:
try:
    responses = pickle.load( open("../../data/Conversations/TCP/pickle/TCP_pairs_Aday.pkl", "rb" ))
except:
    responses = []

## Generación de comandos de paquetes conversaciones TCP

In [68]:
summaries_per_request = 1 # por que normalmente solo querremos generar una conversación

In [82]:
for i in range(1):
    query=query1
    print("...........................................")
    print( " GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter " +str(i+1) +")")
    

    ## Luego seleccionas aleatoriamente resumenes ya generados para pedirle que te saque el comando a partir del paquete
    summaries = random.choices(Conv_summary,k=summaries_per_request)
    text_sum = ""
    
    for summary in summaries:
        query += "This is a conversation summary:\n"
        query += summary + "\n"
        text_sum += summary + "\n"
        query += "\nGenerate python code for creating the packets in the conversation with scapy framework and put them all in a list named pkt_list."

        #Query es el mensaje que le envias al modelo
        print(query)

        #Esta es la petición que le mandas al modelo de IA
        completion = openai.completions.create(
            model="gpt-3.5-turbo-instruct", 
            prompt=query,
            max_tokens=2600,
            temperature=0.1)

    
    print("...Generated!")
    
    # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
    responses.append(response(text_sum, completion))
    
    # Guardamos tanto la petición como la respuesta en un archivo
    pickle.dump(responses, open( "../../data/Conversations/TCP/pickle/TCP_pairs_Aday.pkl", "wb" ) )
    
    print("Number of completitions done: " + str(len(responses)))

...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 1)
This is a IPv4 TCP conversation summary:
Source: IP="193.24.227.230", port=40301, Window: 4128 // Destination: IP="9.9.9.9", port=53, Window: 256// Others: 

This is the python code to generate the 3 way hanshake packets of the conversation, respectively:
scapy.IP(src="193.24.227.230", dst="9.9.9.9")/scapy.TCP(sport=40301, dport=53, flags="S", seq=x, window=4128)
scapy.IP(src="9.9.9.9", dst="193.24.227.230")/scapy.TCP(sport=53, dport=40301, flags="SA", seq=x+1, ack=y, window=256)
scapy.IP(src="193.24.227.230", dst="9.9.9.9")/scapy.TCP(sport=40301, dport=53, flags="SA", seq=y+1, ack=x+1, window=4128)

This part between the handshake and the teardown must be filled with bidirectional realistic TCP traffic, here is an example of these packets:
scapy.IP(src="193.24.227.230", dst="9.9.9.9")/scapy.TCP(sport=40301, dport=53, flags="PA", seq=y+1, ack=x+1, window=8192)/payload1
scapy.

In [84]:
# Este es el resumen de lo que está en el struct de responses para la última petición
print(responses[-1].prompt_summary)
print(responses[-1].completion.choices[0].text)

Source: IP="192.168.0.100", port=8080, Window: 32768 // Destination: IP="192.168.0.101", port=2359, Window: 65536 // Others: 



import scapy.all as scapy

# Generate the 3-way handshake packets
pkt1 = scapy.IP(src="192.168.0.100", dst="192.168.0.101")/scapy.TCP(sport=8080, dport=2359, flags="S", seq=1000, window=32768)
pkt2 = scapy.IP(src="192.168.0.101", dst="192.168.0.100")/scapy.TCP(sport=2359, dport=8080, flags="SA", seq=2000, ack=1001, window=65536)
pkt3 = scapy.IP(src="192.168.0.100", dst="192.168.0.101")/scapy.TCP(sport=8080, dport=2359, flags="A", seq=1001, ack=2001, window=32768)

# Generate bidirectional realistic TCP packets
payload1 = "This is the first payload"
payload2 = "This is the second payload"
payload3 = "This is the third payload"
payload4 = "This is the fourth payload"
payload5 = "This is the fifth payload"

pkt4 = scapy.IP(src="192.168.0.100", dst="192.168.0.101")/scapy.TCP(sport=8080, dport=2359, flags="PA", seq=1001, ack=2001, window=32768)/payload1
pkt5 = sca

### Generación de paquetes

In [87]:
# Ejecutamos los comandos que devuelve el modelo para generar los paquetes
exec(responses[-1].completion.choices[0].text)
print(pkt_list)

[<IP  frag=0 proto=tcp src=192.168.0.100 dst=192.168.0.101 |<TCP  sport=8080 dport=2359 seq=1000 flags=S window=32768 |>>, <IP  frag=0 proto=tcp src=192.168.0.101 dst=192.168.0.100 |<TCP  sport=2359 dport=8080 seq=2000 ack=1001 flags=SA window=65536 |>>, <IP  frag=0 proto=tcp src=192.168.0.100 dst=192.168.0.101 |<TCP  sport=8080 dport=2359 seq=1001 ack=2001 flags=A window=32768 |>>, <IP  frag=0 proto=tcp src=192.168.0.100 dst=192.168.0.101 |<TCP  sport=8080 dport=2359 seq=1001 ack=2001 flags=PA window=32768 |<Raw  load='This is the first payload' |>>>, <IP  frag=0 proto=tcp src=192.168.0.101 dst=192.168.0.100 |<TCP  sport=2359 dport=8080 seq=2001 ack=1026 flags=PA window=65536 |<Raw  load='This is the second payload' |>>>, <IP  frag=0 proto=tcp src=192.168.0.100 dst=192.168.0.101 |<TCP  sport=8080 dport=2359 seq=1026 ack=2027 flags=PA window=32768 |<Raw  load='This is the third payload' |>>>, <IP  frag=0 proto=tcp src=192.168.0.101 dst=192.168.0.100 |<TCP  sport=2359 dport=8080 seq=202

In [89]:
#Enviarlos para probar si se envian correctamente
send(pkt_list)

error: 'H' format requires 0 <= number <= 65535

### Introducción en un pcap

In [93]:
# Guardamos los paquetes generados en un pcap
with open("../../data/Conversations/TCP/pcap/TCPgenerated_3_5_turbo-instruct_conv.pcap", "ba+") as f:
    wrpcap(f, pkt_list,append=True)